## Data Preparation For Description

In [2]:
import pandas as pd
import torch

products = ['table','cell+phone','laptop','keyboard','headphones','chair',
            'lamp','fridge','mouse','printer','tablet','television',
            'vacuum','digital+camera','speaker','iron']

df = pd.DataFrame()
for product in products:
  df_product = pd.read_csv(product+'_Text.csv')
  df_product['Label'] = df_product.shape[0] * [product]
  df = pd.concat([df,df_product])

df.columns = ['Description', 'Label']
df

,Description,Label
0,[ Item Dimensions : 13.4L x 11.5W x 22.8H inc...,table
1,[ Simple stylish design with ample storage an...,table
2,[ The minimalist design desk serves good purp...,table
3,[ Card table with vinyl playing surface mater...,table
4,"[ Simple stylish design, Functional and suita...",table
...,...,...
65,[],iron
66,[ MULTI DIRECTIONAL IRONING – The streamlined...,iron
67,[ THICKER STAINLESS STEEL TANK - All new stai...,iron
68,[ MORE POWER: Classic Steam Iron comes equipp...,iron


In [3]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [4]:
from sentence_transformers import SentenceTransformer
tokenizer = SentenceTransformer('sentence-transformers/use-cmlm-multilingual')
#embeddings = model.encode(sentences)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [5]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

text = tokenizer.encode(df['Description'].tolist())

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
Labels_text = label_encoder.fit_transform(df['Label'].tolist())


## Data Preparation For Image

In [6]:
from google.colab import drive
from torch import nn
import torch
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


path = "./drive/MyDrive/VietHung_PBL6_demo/Data_Image"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

Image_dataset = datasets.ImageFolder(root=path, transform=transform)


FileNotFoundError: [Errno 2] No such file or directory: './drive/MyDrive/VietHung_PBL6_demo/Data_Image'

In [ ]:
import matplotlib.pyplot as plt
for (X1, y1) in (Image_dataset):
  plt.imshow(X1.permute(1, 2, 0))
  print(X1.shape)
  break

In [ ]:
image = []
Labels_image = []
for (X, y) in Image_dataset:
  image.append(X)
  Labels_image.append(y)

In [ ]:
import numpy as np
Data_Fusion = []
#Image
image = np.array(image)
Labels_image = np.array(Labels_image)
#Text
text = np.array(text)
Labels_text = np.array(Labels_text)


for i in range(len(products)):
  for X1,y1,X2,y2 in zip(image[Labels_image==i],Labels_image[Labels_image==i],text[Labels_text==i],Labels_text[Labels_text==i]):
    Data_Fusion.append((X1,X2,i))
    print(X1.shape,X2.shape,i)


In [ ]:
len(Data_Fusion)

In [ ]:
train_dataset, test_dataset = random_split(Data_Fusion, [0.7, 0.3])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
from torchvision.models import resnet18,vgg16

model = resnet18(pretrained=True)

# for param in model.parameters():
# 	param.requires_grad = False

# modelOutputFeats = model.fc.in_features
model.fc


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 68.2MB/s]


Linear(in_features=512, out_features=1000, bias=True)

In [ ]:
import torch.nn as nn

class Vision_model(nn.Module):
    def __init__(self, output_features=128):
        super(Vision_model, self).__init__()

        self.resnet18 = resnet18(pretrained=False)

        modelOutputFeats = self.resnet18.fc.in_features

        self.resnet18.fc = nn.Linear(modelOutputFeats,output_features)
    def forward(self, x):

        x = self.resnet18(x)
        return x


In [ ]:
import torch.nn as nn

class Language_model(nn.Module):
    def __init__(self, input_size=768, output_features=128):
        super(Language_model, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_size, 256,bias=True),
            nn.ReLU(),
            nn.Linear(256, output_features,bias=True),
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
class Multimodal(nn.Module):
    def __init__(self, Vision_model,Language_model,num_classes,output_features=128):
        super(Multimodal , self).__init__()
        self.vision_model = Vision_model()
        self.language_model = Language_model()

        self.encoder_layer = nn.TransformerEncoderLayer(d_model=256, nhead=8)

        self.fc1 = nn.Linear(output_features*2, 128,bias=True)
        self.fc2 = nn.Linear(128, num_classes,bias=True)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, X1, X2):
        X1 = self.vision_model(X1)
        X2 = self.language_model(X2)

        x = torch.cat([X1,X2],dim=1)
        # transformer_encoder = nn.TransformerEncoder(self.encoder_layer,num_layers=1)
        # x = transformer_encoder(x)

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
num_classes = len(Image_dataset.classes)
multimodal = Multimodal(Vision_model,Language_model,num_classes)

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn):
    train_loss, train_acc = 0, 0

    for batch, (X1, X2, y) in enumerate(data_loader):
        print(X1.shape, X2.shape, y.shape)
        # 1. Forward pass
        y_pred = model(X1,X2)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        print(loss,y.shape,y_pred.shape)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for (X1, X2, y) in data_loader:
            # 1. Forward pass
            test_pred = model(X1,X2)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(multimodal.parameters(), lr=0.0001)
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
torch.manual_seed(42)

epochs = 30
for epoch in range(epochs):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_loader,
        model=multimodal,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )
    test_step(data_loader=test_loader,
        model=multimodal,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )


Epoch: 0
---------
torch.Size([32, 3, 224, 224]) torch.Size([32, 768]) torch.Size([32])
tensor(2.7543, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32, 768]) torch.Size([32])
tensor(2.7399, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32, 768]) torch.Size([32])
tensor(2.7024, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32, 768]) torch.Size([32])
tensor(2.7439, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32, 768]) torch.Size([32])
tensor(2.7022, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32, 768]) torch.Size([32])
tensor(2.7357, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32, 768]) torch.Size([32])
tensor(2.640